In [6]:
from PIL import Image 
import requests 
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_id = "/home/dell/disk1/Jinlong/Phi3.5-vision" 

# Note: set _attn_implementation='eager' if you don't have flash_attn installed
model = AutoModelForCausalLM.from_pretrained(
  model_id, 
  device_map="cuda", 
  trust_remote_code=True, 
  torch_dtype="auto", 
#   _attn_implementation='flash_attention_2'    
  _attn_implementation='eager'    
)

# for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
processor = AutoProcessor.from_pretrained(model_id, 
  trust_remote_code=True, 
  num_crops=4
) 

images = []
placeholder = ""

# Note: if OOM, you might consider reduce number of frames in this example.
# for i in range(1,20):
#     url = f"https://image.slidesharecdn.com/azureintroduction-191206101932/75/Introduction-to-Microsoft-Azure-Cloud-{i}-2048.jpg" 
#     images.append(Image.open(requests.get(url, stream=True).raw))
#     placeholder += f"<|image_{i}|>\n"

# messages = [
#     {"role": "user", "content": placeholder+"Summarize the deck of slides."},
# ]

image_path = '/home/dell/disk1/Jinlong/Phi3.5-vision/images/' # load image data

for i in range(1, 4):
    # Construct the local file path for each image
    file_path = f"{image_path}Introduction-to-Microsoft-Azure-Cloud-{i}-2048.jpg"

    # Open and append the image to the list
    img = Image.open(file_path)
    images.append(img)

    # Append the placeholder string
    placeholder += f"<|image_{i}|>\n"
    

messages = [
    {"role": "user", "content": placeholder +  "Summarize the deck of slides."},
]

prompt = processor.tokenizer.apply_chat_template(
  messages, 
  tokenize=False, 
  add_generation_prompt=True
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 1000, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, 
  eos_token_id=processor.tokenizer.eos_token_id, 
  **generation_args
)

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, 
  skip_special_tokens=True, 
  clean_up_tokenization_spaces=False)[0] 

print(response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/dell/disk1/data/Software/Anaconda/envs/JLtorch/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:510: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/dell/disk1/data/Software/Anaconda/envs/JLtorch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.

To encapsulate, the slides feature the following segments:

- Introduction: 
The presentation introduces Microsoft Azure, a cloud computing platform by Microsoft. It highlights the platform's three main types: Hyper-scale, Enterprise, and Hybrid.

- Azure Overview: 
Azure is described as a continually expanding set of cloud services that help organizations meet current and future business challenges. It offers the freedom to build, manage, and deploy applications on a massive global network using favorite tools and frameworks.

- Cloud Computing Services: 
The presentation outlines three types of cloud computing services provided by Azure: Infrastructure-as-a-Service (IaaS) with a focus on hosting, Platform-as-a-Service (PaaS) with a focus on building, and Software-as-a-Service (SaaS) with a focus on consumption.
